_inserts one my fav kdot lyric_

**Kendrick Lamar**

Importing necessary libraries

In [2]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

Establishing a Connection with the Spotify API

In [3]:
current_dir = os.path.dirname(os.path.abspath("Kendrick Lamar"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

# Read the config file
config.read(config_path)
            
client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

Function to monitor duration of scrape

In [4]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

Artist Profile harvesting

In [12]:
from spotipy.oauth2 import SpotifyClientCredentials

# Get credentials from the config file
client_id= '31d85b3f430e45caba7b97174cb75137'
client_secret = '5c6b719bcd8743b0ad1f8cd217ad1e1c'
client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
def collect_spotify_data():
    start_time = timer()
    data = []  # Using a list of dictionaries to collect data

    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist: Kendrick Lamar', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                # Get audio features for track
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    # Handle case where audio features are not available
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                # Get featured artists
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5)  # Retry after a short delay
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break  # Break the loop if there's a persistent error

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print("Elapsed time:", timekeeper(elapsed_time))
    return df_raw

# Collecting data from Spotify API 
df_spotify = collect_spotify_data()

Timeout error: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5). Retrying in 5 seconds...
Elapsed time: 6 minutes, 45 seconds


In [8]:
df_spotify.head()

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Nipsey Hussle,Dedication,1zRTFvd3dpzlr75zVTKf6O,Visionary Artists,2qYP3N8Q23coynyjw18rcg,2024-03-12,0,True,0.513,245813.0,...,-5.448,0.0,0.4320,0.43100,0.0,0.2260,0.170,94.527,4.0,[Kendrick Lamar]
1,Kendrick Lamar,Not Like Us,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,2024-05-04,90,True,0.898,274192.0,...,-7.001,1.0,0.0776,0.01070,0.0,0.1410,0.214,101.061,4.0,[]
2,Various Artists,Alan Anderson - “Levitate” by Kendrick Lamar,4u3pOE7VcP0LX7HhugS0yc,LA Clippers - Cali Fire,5xIwaxMrnDHJwVWwXhiu4W,2016-10-31,0,False,0.000,6000.0,...,-7.246,1.0,0.0000,0.98200,0.0,0.0000,0.000,0.000,0.0,[]
3,Kendrick Lamar,Money Trees,2HbKqm4o0w5wEeEFXm2sD4,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,77,True,0.716,386907.0,...,-7.355,1.0,0.1220,0.07030,0.0,0.2240,0.344,71.994,4.0,[Jay Rock]
4,Kendrick Lamar,DNA.,6HZILIRieu8S0iqY8kIKhj,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,79,True,0.638,185947.0,...,-6.664,1.0,0.3570,0.00454,0.0,0.0842,0.422,139.913,4.0,[]


Filtering the artist name column to contain only 'Kendrick Lamar'

In [9]:
kdot = df_spotify[df_spotify['artist_name']=='Kendrick Lamar']
kdot.head()

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
1,Kendrick Lamar,Not Like Us,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,2024-05-04,90,True,0.898,274192.0,...,-7.001,1.0,0.0776,0.01070,0.0,0.1410,0.214,101.061,4.0,[]
3,Kendrick Lamar,Money Trees,2HbKqm4o0w5wEeEFXm2sD4,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,77,True,0.716,386907.0,...,-7.355,1.0,0.1220,0.07030,0.0,0.2240,0.344,71.994,4.0,[Jay Rock]
4,Kendrick Lamar,DNA.,6HZILIRieu8S0iqY8kIKhj,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,79,True,0.638,185947.0,...,-6.664,1.0,0.3570,0.00454,0.0,0.0842,0.422,139.913,4.0,[]
5,Kendrick Lamar,LOVE. FEAT. ZACARI.,6PGoSes0D9eUDeeAafB2As,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,81,True,0.800,213400.0,...,-7.343,1.0,0.0924,0.26400,0.0,0.1530,0.779,126.058,4.0,[Zacari]
6,Kendrick Lamar,Alright,3iVcZ5G6tvkXZkZKlMpIUs,To Pimp A Butterfly,7ycBtnsMtyVbbwTfJwRjSP,2015-03-16,79,True,0.796,219333.0,...,-5.974,1.0,0.2380,0.07420,0.0,0.0827,0.558,110.034,4.0,[]


In [10]:
kdot['album_name'].unique()

array(['Not Like Us', 'good kid, m.A.A.d city', 'DAMN.',
       'To Pimp A Butterfly', 'meet the grahams', 'Section.80',
       'Mr. Morale & The Big Steppers', 'good kid, m.A.A.d city (Deluxe)',
       'i', 'untitled unmastered.', 'Overly Dedicated', 'Alright',
       'DAMN. COLLECTORS EDITION.', 'untitled 07 | levitate', 'Top 30 US',
       'The Heart Part 5', 'Gangsta Bangaz Vol. 1', 'Rainy Day R&B',
       'Rap Gods', 'Smokey Romantic',
       'The Hate U Give (Original Motion Picture Soundtrack)',
       'College Party Anthems', 'Best Of 2017 Hip Hop',
       'Hip Hop Hit List (Vol. 2)', 'Gangsta Bangas 2', 'We Run L.A.',
       'West Coast Hip-Hop', 'LoFi Chillhop',
       'Hip Hop Hit List (Vol. IV)'], dtype=object)

Categorizing the extract according to what type of release

In [11]:
studio_albums = ['Section.80',
                 'good kid, m.A.A.d city',
                 'To Pimp A Butterfly',                 
                 'DAMN.',
                 'DAMN. COLLECTORS EDITION.', 
                 'Mr. Morale & The Big Steppers']

compilation_album = ['untitled unmastered.']

deluxe_album = ['good kid, m.A.A.d city (Deluxe)']

mixtape = ['Overly Dedicated']

singles = kdot['album_name'] == kdot['track_name']
album_condition = kdot['album_name'].isin(studio_albums)
compilations = kdot['album_name'].isin(compilation_album)
deluxe = kdot['album_name'].isin(deluxe_album)
mixtape_mapping = kdot['album_name'].isin(mixtape)

conditions = [singles, album_condition, compilations, deluxe, mixtape_mapping]

choices = ['Single', 'Studio Album', 'Compilation', 'Deluxe Album', 'Mixtape']

kdot['type'] = np.select(conditions, choices, default= '')

kdot[['artist_name', 'track_name', 'album_name', 'type']].head()

C:\Users\HP\AppData\Local\Temp\ipykernel_16884\726595935.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kdot['type'] = np.select(conditions, choices, default= '')


,artist_name,track_name,album_name,type
1,Kendrick Lamar,Not Like Us,Not Like Us,Single
3,Kendrick Lamar,Money Trees,"good kid, m.A.A.d city",Studio Album
4,Kendrick Lamar,DNA.,DAMN.,Studio Album
5,Kendrick Lamar,LOVE. FEAT. ZACARI.,DAMN.,Studio Album
6,Kendrick Lamar,Alright,To Pimp A Butterfly,Studio Album


scraping a spotify playlist that contains all Kendrick's features 

In [13]:
# start time
start_time = timer()

# Get playlist URI
playlist_uri = "https://open.spotify.com/playlist/5Z7uz9iDMfINUocPZbgHRG?si=8JTn4Ae-SEy6SCEYSjIm6w"

# Get playlist tracks
playlist = sp.playlist(playlist_uri)
tracks = playlist['tracks']['items']

# Initialize data structures
song_features = []

# Define offset for pagination
offset = 0
limit = 100

# Iterate through playlist tracks, making additional requests as needed
while True:
    # Get playlist tracks with pagination
    results = sp.user_playlist_tracks(user="George Pacitto", playlist_id=playlist_uri, offset=offset, limit=limit)

    # Check if there are more tracks to retrieve
    if not results['items']:
        break

    # Extract track information and audio features
    for track in results['items']:
        track_uri = track['track']['uri']
        track_name = track['track']['name']

        # Extract track information
        track_info = sp.track(track_uri)
        artist_name = track_info['artists'][0]['name']
        track_name = track_info['name']
        track_id = track_info['id']
        album_name = track_info['album']['name']
        album_id = track_info['album']['id']
        release_date = track_info['album']['release_date']
        duration_ms = track_info['duration_ms']
        popularity = track_info['popularity']
        explicit = track_info['explicit']
        
        
        # Extract audio features
        track_features = sp.audio_features(track_uri)

        # Check if audio features are available
        if track_features:
            track_features = track_features[0]

            # Extract audio features
            danceability = track_features['danceability']
            energy = track_features['energy']
            key = track_features['key']
            loudness = track_features['loudness']
            mode = track_features['mode']
            speechiness = track_features['speechiness']
            acousticness = track_features['acousticness']
            instrumentalness = track_features['instrumentalness']
            liveness = track_features['liveness']
            valence = track_features['valence']            
            tempo = track_features['tempo']
            time_signature = track_features['time_signature']
            
            # get featured artists
            featured_artists = []
            if len(track_info['artists']) > 1:
                feat_artists = []
                for j in range(1, len(track_info['artists'])):
                    feat_artists.append(track_info['artists'][j]['name'])
                featured_artists.append(feat_artists)
            else:
                featured_artists.append([])
   

            # Store data
            song_features.append([
                artist_name, track_name, track_id, album_name,
                album_id, release_date, duration_ms, popularity,
                explicit, danceability, energy, key, loudness,
                mode, speechiness, acousticness, instrumentalness,
                liveness, valence, tempo, time_signature, featured_artists
            ])
        else:
            print("Audio features not available for track:", track_name)

    # Update offset for next request
    offset += limit

# Save data to CSV file
ft = pd.DataFrame(
    song_features, 
    columns=[ 'artist_name', 'track_name', 'track_id', 'album_name', 'album_id',
                'release_date', 'duration_ms', 'popularity', 'explicit',
                'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                'acousticness', 'instrumentalness', 'liveness',
                'valence', 'tempo', 'time_signature', 'featured_artists'])

# end time and print
end_time = timer()
elapsed_time = int(end_time - start_time)
print(f"Elapsed time:", timekeeper(elapsed_time))

Elapsed time: 1 minutes, 11 seconds


In [119]:
ft.head()

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists,type
0,8 Ounz,College Girls (feat. Kendrick Lamar),5MDvjyhIVrl7Lt8z0YEflo,"Hip Hop Had a Dream: The World Wide Tape, Vol. 2",6zBqDiLjTsqFdzYTusNDCn,2012-03-05,273737,11,True,0.788,...,0,0.243,0.2460,0.000000,0.7450,0.757,93.965,4,[[]],Feature
1,Jay Rock,Hood Gone Love It (feat. Kendrick Lamar),6i6cWTCYBOKEdJnWjzm2yk,Follow Me Home,6bPbKuFNW6Vausf1PExvd9,2011-07-26,245466,67,True,0.636,...,1,0.312,0.0879,0.000000,0.2890,0.677,91.810,4,[[Kendrick Lamar]],Feature
2,Lyn Charles,Star Life (feat. Kendrick Lamar),2ZblN6mtRfOB3sS8Z8bLY6,Star Life (feat. Kendrick Lamar),3acn5bSSEvwxhrsmti72fg,2012-03-04,209059,20,False,0.789,...,0,0.258,0.0451,0.000001,0.3140,0.412,122.152,5,[[]],Feature
3,Birdman,B-Boyz,5MpIrad80ZF5rSPbovUfsZ,Kiss The Ring (Deluxe),5brUmg8Gox3t2OmAQzLivJ,2012-01-01,297160,0,True,0.491,...,1,0.301,0.0219,0.000000,0.0698,0.445,96.563,5,"[[Mack Maine, Kendrick Lamar, Ace Hood, DJ Kha...",Feature
4,Talib Kweli,Push Thru,77eyE4gfO6iVtbD3CoqblG,Prisoner of Conscious,7kAg6DbRZXNLKzvQAdYkJp,2013-05-07,298255,0,True,0.372,...,1,0.170,0.0676,0.000002,0.0757,0.453,79.784,4,"[[Curren$y, Kendrick Lamar, Glen Reynolds]]",Feature


Creating a 'type' field for the ft DataFrame and categorizing everything as a 'Feature' before concatenating it with the kdot DataFrame

In [120]:
ft['type']='Feature' 

df = pd.concat([kdot, ft], ignore_index= True, sort= False)

Drop rows from the _type_ field with unassigned values

In [121]:
notype = df[df['type'] =='']
notype[['artist_name', 'track_name', 'album_name', 'type']]

,artist_name,track_name,album_name,type
112,Kendrick Lamar,LOVE.,Top 30 US,
141,Kendrick Lamar,War Is My Love,Gangsta Bangaz Vol. 1,
155,Kendrick Lamar,PRIDE.,Rainy Day R&B,
157,Kendrick Lamar,"Bitch, Don’t Kill My Vibe",Rap Gods,
171,Kendrick Lamar,HOC,We Run L.A.,
172,Kendrick Lamar,LOVE.,Hip Hop Hit List (Vol. 3),
173,Kendrick Lamar,You Don't Like Me,"Beaches & Bridges - La 2 da Bay, Vol. 5",
174,Kendrick Lamar,DNA.,The Hate U Give (Original Motion Picture Sound...,
175,Kendrick Lamar,m.A.A.d city,West Coast Hip-Hop,
180,Kendrick Lamar,FEEL.,LoFi Chillhop,


In [123]:
notype = df[df['type'] =='']
flush = notype.index
df = df.drop(flush)
notype[['artist_name', 'track_name', 'album_name', 'type']]

,artist_name,track_name,album_name,type


In [129]:
df = df.sort_values(by= ['album_name', 'popularity'], ascending= [True, False])
# df.drop_duplicates(subset= 'track_name', keep= 'first', inplace= True, ignore_index= True)
df

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists,type,duration
273,Terrace Martin,Triangle Ship (feat. Kendrick Lamar),1TycxSppuyhz7NVbiOs870,3ChordFold,3jVyQ0ynHkn1fq1BaSHsw8,2013-08-13,20,False,0.357,0.470,...,0.3830,0.0516,0.000000,0.1180,0.253,171.347,4,[[]],Feature,05:40
283,Jay Rock,Easy Bake (feat. Kendrick Lamar & SZA),2iXm6lM5KNUHOGP5OQSeyA,90059,1fsvlnonD95p6VTEdQ7zJn,2015-09-11,38,True,0.566,0.586,...,0.8270,0.3570,0.000000,0.4250,0.619,174.262,4,"[[Kendrick Lamar, SZA]]",Feature,04:51
359,Beyoncé,AMERICA HAS A PROBLEM (feat. Kendrick Lamar),5xlLG6RthtyXShVhGA3ojN,AMERICA HAS A PROBLEM (feat. Kendrick Lamar),2WMGh4pKjsJ21Jlj1DzT7c,2023-05-19,44,False,0.788,0.694,...,0.0372,0.0110,0.000015,0.1420,0.229,126.042,4,[[Kendrick Lamar]],Feature,04:20
214,Funkadelic,Ain't That Funkin' Kinda Hard on You? (feat. K...,6SgQIoLn8kpu8J4wfwGWy8,Ain't That Funkin' Kinda Hard on You? (We Ain'...,6oeY3dvDGQ5hJxQNHxA8dS,2016-03-09,43,True,0.635,0.874,...,0.1760,0.1560,0.000000,0.2450,0.632,89.895,4,"[[Kendrick Lamar, Ice Cube]]",Feature,05:09
244,Tech N9ne,I Love Music (Kendrick Lamar & Oobergeek),4prHK0UHZtN5Fvlr6PFcV7,All 6's And 7's,2XFw84i6R3GcuxhppaRyQW,2011-06-07,15,True,0.609,0.858,...,0.3420,0.1980,0.000000,0.2860,0.844,103.544,5,[[]],Feature,03:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Kendrick Lamar,untitled 06 | 06.30.2014.,2b8EF90yqJUNBrle61FnjO,untitled unmastered.,2BV6bwYo6UZXHlrzQtPjXC,2016-03-04,21,False,0.653,0.548,...,0.0417,0.0973,0.000007,0.2460,0.641,88.788,4,[],Compilation,03:28
163,Kendrick Lamar,untitled 03 | 05.28.2013.,3pAlST8aex1y0HUNiv2KO9,untitled unmastered.,2BV6bwYo6UZXHlrzQtPjXC,2016-03-04,18,False,0.829,0.544,...,0.2410,0.5630,0.000002,0.1080,0.572,103.977,4,[],Compilation,02:34
167,Kendrick Lamar,untitled 07 | 2014 - 2016,5Vry4BTGuNflhlrrWyn0YR,untitled unmastered.,2BV6bwYo6UZXHlrzQtPjXC,2016-03-04,17,False,0.536,0.451,...,0.5790,0.7260,0.000000,0.6940,0.357,99.978,4,[],Compilation,08:16
168,Kendrick Lamar,untitled 01 | 08.19.2014.,7jel3YgcqfD2GGpesoHJvv,untitled unmastered.,2BV6bwYo6UZXHlrzQtPjXC,2016-03-04,15,False,0.574,0.571,...,0.3980,0.3310,0.000000,0.0995,0.111,102.160,4,[],Compilation,04:07


Modifying the 'mode' column by replacing 0 with Minor and 1 with Major

In [124]:
df['mode'] = df['mode'].replace({0:'Minor', 1:'Major'})
df[['track_name', 'album_name', 'mode']].head()

,track_name,album_name,mode
0,Not Like Us,Not Like Us,Major
1,Money Trees,"good kid, m.A.A.d city",Major
2,DNA.,DAMN.,Major
3,Alright,To Pimp A Butterfly,Major
4,LOVE. FEAT. ZACARI.,DAMN.,Major


Normalizing the _duration ms_ column to appear as minutes and seconds 

In [125]:
df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')

# extract the minutes and seconds components of the duration as strings
df['duration'] = df['duration_ms'].dt.components['minutes'].astype(str).str.zfill(2) + ':' + df['duration_ms'].dt.components['seconds'].astype(str).str.zfill(2)
df.drop(columns=['duration_ms'], inplace= True)
df[['track_name','duration']].head()

,track_name,duration
0,Not Like Us,04:34
1,Money Trees,06:26
2,DNA.,03:05
3,Alright,03:39
4,LOVE. FEAT. ZACARI.,03:33


Creating three new fields: year, month and day a song was released from release date column

In [ ]:
...

In [128]:
df[df['album_name']=='good kid, m.A.A.d city']

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists,type,duration
1,Kendrick Lamar,Money Trees,2HbKqm4o0w5wEeEFXm2sD4,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,78,True,0.716,0.531,...,0.1220,0.070300,0.000000,0.224,0.3440,71.994,4,[Jay Rock],Studio Album,06:26
6,Kendrick Lamar,"Bitch, Don’t Kill My Vibe",4Pwjz3DfvfQWV0rO2V8jyh,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,69,True,0.587,0.698,...,0.0943,0.015200,0.000000,0.442,0.0985,133.954,4,[],Studio Album,05:10
9,Kendrick Lamar,m.A.A.d city,439TlnnznSiBbQbgXiBqAd,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,66,True,0.487,0.729,...,0.2710,0.053800,0.000004,0.440,0.2170,91.048,4,[MC Eiht],Studio Album,05:50
54,Kendrick Lamar,Backseat Freestyle,6fAyRZ0GHuuvSOEIOwi58N,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,52,True,0.546,0.651,...,0.3000,0.000739,0.000000,0.235,0.6460,77.878,4,[],Studio Album,03:32
62,Kendrick Lamar,Real,7pdCmGogZHZH3Xv7LnqVrH,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,45,True,0.652,0.458,...,0.1940,0.028200,0.000000,0.664,0.1740,100.046,4,[Anna Wise],Studio Album,07:23
65,Kendrick Lamar,Sherane a.k.a Master Splinter’s Daughter,7zdORPbuIBkZHCJXl9J2e5,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,42,True,0.503,0.508,...,0.5200,0.486000,0.010200,0.124,0.3170,81.908,4,[],Studio Album,04:33
68,Kendrick Lamar,"Sing About Me, I'm Dying Of Thirst",0sd6BRTa0O96tfEbFGhJF9,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,49,True,0.654,0.753,...,0.3950,0.303000,0.000081,0.130,0.3710,110.722,5,[],Studio Album,12:03
69,Kendrick Lamar,The Art of Peer Pressure,2PieESXphabz5AkALFS08X,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,45,True,0.555,0.611,...,0.3560,0.286000,0.001060,0.273,0.4110,94.939,4,[],Studio Album,05:24
71,Kendrick Lamar,m.A.A.d city,0wagV3icLdCE7uP7rjyOfY,"good kid, m.A.A.d city",0E47vczx9XVds1Lr2Y8rze,2012,42,False,0.462,0.676,...,0.2150,0.063000,0.000012,0.330,0.1620,74.637,4,[MC Eiht],Studio Album,05:50
77,Kendrick Lamar,Money Trees,16SIvLDCFALvBDTmepJMIA,"good kid, m.A.A.d city",0E47vczx9XVds1Lr2Y8rze,2012,47,False,0.715,0.537,...,0.1070,0.067100,0.000000,0.208,0.3600,72.001,4,[Jay Rock],Studio Album,06:26
